In [1]:
import pandas as pd
from pathlib import Path
import psycopg2
import numpy as np

In [2]:
sofa_csv = Path('./data/pivoted_sofa.csv')

In [3]:
sofa = pd.read_csv(sofa_csv)
print(sofa.shape)
print(sofa.dtypes)
sofa.head()

(5407454, 29)
icustay_id                  int64
hr                          int64
starttime                  object
endtime                    object
pao2fio2ratio_novent      float64
pao2fio2ratio_vent        float64
rate_epinephrine          float64
rate_norepinephrine       float64
rate_dopamine             float64
rate_dobutamine           float64
meanbp_min                float64
gcs_min                   float64
urineoutput               float64
bilirubin_max             float64
creatinine_max            float64
platelet_min              float64
respiration               float64
coagulation               float64
liver                     float64
cardiovascular            float64
cns                       float64
renal                     float64
respiration_24hours         int64
coagulation_24hours         int64
liver_24hours               int64
cardiovascular_24hours      int64
cns_24hours                 int64
renal_24hours               int64
sofa_24hours                int64


,icustay_id,hr,starttime,endtime,pao2fio2ratio_novent,pao2fio2ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,...,cardiovascular,cns,renal,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,sofa_24hours
0,200001,0,2181-11-25 18:00:00,2181-11-25 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,1,0,0,0,2,3
1,200001,1,2181-11-25 19:00:00,2181-11-25 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,0,1,0,0,0,2,3
2,200001,2,2181-11-25 20:00:00,2181-11-25 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0,1,0,0,0,2,3
3,200001,3,2181-11-25 21:00:00,2181-11-25 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0,1,0,0,0,2,3
4,200001,4,2181-11-25 22:00:00,2181-11-25 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,3.0,0,1,0,0,0,3,4


In [4]:
con = psycopg2.connect(dbname='eicu', user='sudarshan', host='/var/run/postgresql')

pats = """
select * from patient;
"""
apache = """
select * from apachepatientresult;
"""

eicu_pats = pd.read_sql(pats, con)
apache_scores = pd.read_sql(apache, con)
con.close()

In [5]:
print(eicu_pats.shape)
print(eicu_pats.dtypes)
eicu_pats.head()

(200859, 29)
patientunitstayid              int64
patienthealthsystemstayid      int64
gender                        object
age                           object
ethnicity                     object
hospitalid                     int64
wardid                         int64
apacheadmissiondx             object
admissionheight              float64
hospitaladmittime24           object
hospitaladmitoffset            int64
hospitaladmitsource           object
hospitaldischargeyear          int64
hospitaldischargetime24       object
hospitaldischargeoffset        int64
hospitaldischargelocation     object
hospitaldischargestatus       object
unittype                      object
unitadmittime24               object
unitadmitsource               object
unitvisitnumber                int64
unitstaytype                  object
admissionweight              float64
dischargeweight              float64
unitdischargetime24           object
unitdischargeoffset            int64
unitdischargelocation    

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
0,141168,128919,Female,70,Caucasian,59,91,"Rhythm disturbance (atrial, supraventricular)",152.4,15:54:00,...,Direct Admit,1,admit,84.3,85.8,03:50:00,3596,Death,Expired,002-34851
1,141178,128927,Female,52,Caucasian,60,83,,162.6,08:56:00,...,Emergency Department,1,admit,54.4,54.4,09:18:00,8,Step-Down Unit (SDU),Alive,002-33870
2,141179,128927,Female,52,Caucasian,60,83,,162.6,08:56:00,...,ICU to SDU,2,stepdown/other,NaN,60.4,19:20:00,2042,Home,Alive,002-33870
3,141194,128941,Male,68,Caucasian,73,92,"Sepsis, renal/UTI (including bladder)",180.3,18:18:40,...,Floor,1,admit,73.9,76.7,15:31:00,4813,Floor,Alive,002-5276
4,141196,128943,Male,71,Caucasian,67,109,,162.6,20:21:00,...,ICU to SDU,2,stepdown/other,NaN,63.2,22:23:00,1463,Floor,Alive,002-37665


In [6]:
eicu_pats['age'].replace('> 89', '90', inplace=True)
eicu_pats['age'].fillna('-1', inplace=True)
eicu_pats['age'] = pd.to_numeric(eicu_pats['age'])
len(eicu_pats[(eicu_pats['age'] != -1) & (eicu_pats['age'] < 16)])

181

In [8]:
print(apache_scores.shape)
print(apache_scores.dtypes)
apache_scores.head()

(297064, 23)
apachepatientresultsid             int64
patientunitstayid                  int64
physicianspeciality               object
physicianinterventioncategory     object
acutephysiologyscore               int64
apachescore                        int64
apacheversion                     object
predictedicumortality             object
actualicumortality                object
predictediculos                  float64
actualiculos                     float64
predictedhospitalmortality        object
actualhospitalmortality           object
predictedhospitallos             float64
actualhospitallos                float64
preopmi                            int64
preopcardiaccath                   int64
ptcawithin24h                      int64
unabridgedunitlos                float64
unabridgedhosplos                float64
actualventdays                   float64
predventdays                     float64
unabridgedactualventdays         float64
dtype: object


,apachepatientresultsid,patientunitstayid,physicianspeciality,physicianinterventioncategory,acutephysiologyscore,apachescore,apacheversion,predictedicumortality,actualicumortality,predictediculos,...,predictedhospitallos,actualhospitallos,preopmi,preopcardiaccath,ptcawithin24h,unabridgedunitlos,unabridgedhosplos,actualventdays,predventdays,unabridgedactualventdays
0,26570,141168,critical care medicine (CCM),Unknown,49,65,IV,2.6988064660976061E-2,EXPIRED,3.038388,...,7.546453,2.4972,0,0,0,2.4972,2.4972,NaN,NaN,NaN
1,26571,141168,critical care medicine (CCM),Unknown,49,65,IVa,2.8889370181248972E-2,EXPIRED,3.091127,...,6.628720,2.4972,0,0,0,2.4972,2.4972,NaN,NaN,NaN
2,53135,141194,critical care medicine (CCM),Unknown,57,70,IV,3.7888052081065979E-2,ALIVE,4.620982,...,13.338449,9.2167,0,0,0,3.3423,9.2167,NaN,NaN,NaN
3,53136,141194,critical care medicine (CCM),Unknown,57,70,IVa,4.6447743452429233E-2,ALIVE,4.167129,...,12.978228,9.2167,0,0,0,3.3423,9.2167,NaN,NaN,NaN
4,8,141203,hospitalist,I,73,90,IVa,0.29160851639408297,ALIVE,8.670299,...,16.319389,3.7493,0,0,0,1.2979,3.7493,2.0,5.738093,2.0
